In [1]:
#loading required libraries

import sys
from pathlib import Path
import json

import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib

import gtfs_kit as gk

In [2]:
#setting GTFS data directory path for loading

DIR = Path('..')
print(DIR)
sys.path.append(str(DIR))
DATA_DIR = DIR/'gtfs_data/'

..


In [3]:
#loading GTFS data and list the feed components

#path = DATA_DIR/'netherlands-gtfs.zip'
path = DATA_DIR/'gtfs-openov-nl.zip'
gk.list_feed(path)

,file_name,file_size
0,agency.txt,2565
1,calendar_dates.txt,3761852
2,feed_info.txt,174
3,routes.txt,149010
4,shapes.txt,127577478
5,stop_times.txt,1002633724
6,stops.txt,2866473
7,trips.txt,65781990


In [4]:
#Describe feed details

feed = gk.read_feed(path, dist_units='km')
feed.describe()

,indicator,value
0,agencies,"[allGo (Keolis), Arriva, Bravo (Arriva), Bravo..."
1,timezone,Europe/Amsterdam
2,start_date,20201217
3,end_date,20210808
4,num_routes,2493
5,num_trips,930228
6,num_stops,46611
7,num_shapes,9155
8,sample_date,20201224
9,num_routes_active_on_sample_date,1974


In [12]:
feed.validate()

KeyboardInterrupt: 

In [4]:
import requests
import shutil
import os
import zipfile

In [5]:
with zipfile.ZipFile('../gtfs_data/gtfs-openov-nl.zip', 'r') as file:
    file.extractall('../gtfs_data/netherlands/')

os.listdir('../gtfs_data/netherlands')

['agency.txt',
 'calendar_dates.txt',
 'feed_info.txt',
 'routes.txt',
 'shapes.txt',
 'stops.txt',
 'stop_times.txt',
 'trips.txt']

In [2]:
import pandas as pd

trips = pd.read_csv('../gtfs_data/netherlands/trips.txt', low_memory=False)
shapes = pd.read_csv('../gtfs_data/netherlands/shapes.txt', low_memory=False)

In [3]:
trips.head()

,route_id,service_id,trip_id,realtime_trip_id,trip_headsign,trip_short_name,trip_long_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,60663,1,123151939,GVB:248:67,Houthavens,NaN,NaN,0,NaN,911191.0,1,NaN
1,60663,1,123151940,GVB:248:70,Houthavens,NaN,NaN,0,NaN,911191.0,1,NaN
2,60663,1,123151932,GVB:248:53,Houthavens,NaN,NaN,0,NaN,911191.0,1,NaN
3,60663,1,123151991,GVB:248:184,Centraal Station,NaN,NaN,1,NaN,911190.0,1,NaN
4,60663,1,123151909,GVB:248:6,Houthavens,NaN,NaN,0,NaN,911191.0,1,NaN


In [4]:
shapes.head()

,shape_id,shape_pt_sequence,shape_pt_lat,shape_pt_lon,shape_dist_traveled
0,899197,1,51.93779,4.33613,0
1,899197,2,51.93312,4.34160,1076
2,899197,4,51.93318,4.34167,1085
3,899197,5,51.93322,4.34171,1090
4,899197,6,51.93330,4.34192,1107


In [5]:
num_trips_by_shape = trips.groupby('shape_id').aggregate({'route_id': 'count'}).reset_index()
num_trips_by_shape.rename({'route_id': 'trip_count'}, axis='columns', inplace=True)

shapes_with_nums = shapes.merge(num_trips_by_shape, on='shape_id')
shapes_with_nums.head()

,shape_id,shape_pt_sequence,shape_pt_lat,shape_pt_lon,shape_dist_traveled,trip_count
0,899197,1,51.93779,4.33613,0,146
1,899197,2,51.93312,4.34160,1076,146
2,899197,4,51.93318,4.34167,1085,146
3,899197,5,51.93322,4.34171,1090,146
4,899197,6,51.93330,4.34192,1107,146


In [6]:
from datashader.utils import lnglat_to_meters
import numpy as np

shapes_with_nums['coord_x'], shapes_with_nums['coord_y'] = \
    lnglat_to_meters(shapes_with_nums['shape_pt_lon'], shapes_with_nums['shape_pt_lat'])

def split_df_by(df, byvar):
    df_parts = []
    for level, df_part in df.groupby(byvar):
        empty = pd.DataFrame([[level if colname == byvar else np.NaN for colname in df.columns]],
                             columns=df.columns)
        df_parts.append(df_part.append(empty))
    return pd.concat(df_parts) 

sep_shapes = split_df_by(shapes_with_nums, 'shape_id')

In [26]:
import bokeh.plotting as bp
from bokeh.models.tiles import WMTSTileSource
import datashader as ds
import datashader.transfer_functions as tf
from datashader.bokeh_ext import InteractiveImage
from datashader.utils import export_image
import colorcet as cc
import os

bp.output_notebook()

x_range=(0.3e6, 1.155e6)
y_range=(4.994e6, 8.052e6)

if not os.path.exists('./img'):
    os.mkdir('./img')
p = bp.figure(tools='pan,wheel_zoom,reset',
              plot_width=int(600),
              plot_height=int(600),
              x_range=x_range,
              y_range=y_range)

p.axis.visible = False
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

url = "https://cartodb-basemaps-b.global.ssl.fastly.net/dark_all/{z}/{x}/{y}.png"
tile_renderer = p.add_tile(WMTSTileSource(url=url))
tile_renderer.alpha = 1

def image_callback(x_range, y_range, w, h, color_fn=tf.shade):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.line(sep_shapes, 'coord_x', 'coord_y', agg=ds.sum('trip_count'))
    image = tf.shade(agg, cmap=cc.fire, how='eq_hist')
    return image

export_image(image_callback(x_range=x_range, y_range=y_range, w=2000, h=2000),
             filename="BUD_schedule_fire", background='black')
InteractiveImage(p, image_callback)

Loading BokehJS ...

C:\Users\srijith\anaconda3\envs\geo_env\lib\site-packages\datashader\bokeh_ext.py:233: VisibleDeprecationWarning: InteractiveImage has been deprecated as of datashader 0.8.0. It is not supported in JupyterLab and Bokeh server environments. Please use the HoloViews datashader integration instead.
  warnings.warn('InteractiveImage has been deprecated as of datashader 0.8.0. '
